# Open Data - Socrata
Seeing if the catalog can be automated

**Note** currently not using an app-token - may want one in the future
https://dev.socrata.com/docs/app-tokens.html


In [15]:
from sodapy import Socrata
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize

In [6]:
# No application token or user identification
client = Socrata("data.edmonton.ca", None)

Want to see if we can automate querying the catalog
http://api.us.socrata.com/api/catalog/v1?domains=data.edmonton.ca&search_context=data.edmonton.ca


In [10]:
# Read a specific dataset
df = pd.read_json("http://data.edmonton.ca/resource/txav-7fn5.json")
df.head(5)

,:@computed_region_da6r_6gkw,amount_fallen,date,date_time,location,station_number,time,year
0,6.0,0.1,2015-06-13,2015-06-13 15:05:39,"{'type': 'Point', 'coordinates': [-113.545, 53...",RG30,15:05:39,2015
1,4.0,0.1,2015-06-04,2015-06-04 16:15:51,"{'type': 'Point', 'coordinates': [-113.32, 53....",RG59,16:15:51,2015
2,10.0,0.1,2015-09-12,2015-09-12 22:03:55,"{'type': 'Point', 'coordinates': [-113.551, 53...",RG57,22:03:55,2015
3,12.0,0.1,2015-07-17,2015-07-17 04:41:29,"{'type': 'Point', 'coordinates': [-113.479, 53...",RG53,04:41:29,2015
4,4.0,0.1,2015-09-06,2015-09-06 01:51:12,"{'type': 'Point', 'coordinates': [-113.373, 53...",RG37,01:51:12,2015


In [21]:
url = 'http://api.us.socrata.com/api/catalog/v1'

params = dict(
    domains='data.edmonton.ca',
    search_context='data.edmonton.ca',
    tags='population'
)

resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

In [35]:
# Now, parse the data
result = json_normalize(data, 'results', ['description'])
result

KeyError: 'description'

Trying to turn this into a regular tabular dataset

In [30]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

In [33]:
flat = flatten_json(result)
#json_normalize(flat)
flat

{'':                                        classification  \
 0   {'categories': ['demographics'], 'tags': [], '...   
 1   {'categories': ['demographics'], 'tags': [], '...   
 2   {'categories': ['demographics'], 'tags': [], '...   
 3   {'categories': ['demographics'], 'tags': [], '...   
 4   {'categories': ['demographics'], 'tags': [], '...   
 5   {'categories': ['demographics'], 'tags': [], '...   
 6   {'categories': ['demographics'], 'tags': [], '...   
 7   {'categories': ['demographics'], 'tags': [], '...   
 8   {'categories': ['demographics'], 'tags': [], '...   
 9   {'categories': ['transportation', 'demographic...   
 10  {'categories': ['demographics', 'finance'], 't...   
 11  {'categories': ['demographics'], 'tags': [], '...   
 12  {'categories': ['demographics', 'environment']...   
 13  {'categories': ['demographics'], 'tags': [], '...   
 14  {'categories': ['economy', 'demographics'], 't...   
 15  {'categories': ['demographics'], 'tags': [], '...   
 16  {'cat